# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [2]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [3]:
Operator_name = driver.find_element_by_name('OperSearch')
Operator_name.send_keys('dirt')
button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

`.text` will help you here.

In [4]:
results = driver.find_elements_by_tag_name('tr')
for result in results:
    print(result.text)

Operator Name or Mine Name
Search  
Abandoned*
Indicates Mine is Abandoned and Sealed
*CM (Coal or Metal Mine/Nonmetal Mine)
C
M ...... Coal
...... Metal/Nonmetal
Abandoned*
Indicates Mine is Abandoned and Sealed
*CM (Coal or Metal Mine/Nonmetal Mine)
C
M ...... Coal
...... Metal/Nonmetal
ID State Operator Mine Name Type CM* Status Commodity More Info
3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
4104757
TX  Dirt Works   Portable #1 Surface M  Intermittent  Construction Sand and Gravel 
0801306
FL  Holley Dirt Company, Inc   River R

### Loop through each operator result, printing its name

You can use list slicing or an `if` statement to skip the non-data row(s).

In [5]:
for result in results[7:27]:
        name = result.find_elements_by_tag_name('td')[2]
        print(name.text)

Newberg Rock & Dirt  
Bender Sand & Dirt  
Dirt Company  
Dirt Doctor Inc  
Dirt Work Specialists LLC  
Dirt Works  
Holley Dirt Company, Inc  
Krueger Brothers Gravel & Dirt  
M R Dirt  
M.R. Dirt Inc.  
P B Dirt Movers, Inc  
P B Dirt Movers, Inc.  
PB Dirt Movers  
Prescott Dirt, LLC  
R D Blankenship Dirt Work LLC  
Sand & Dirt, Inc  
SIMPSON DIRTWORX LLC  
SIMPSON DIRTWORX LLC  
Spry's Dirt & Gravel, Inc.  
Vogt Dirt Service  


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [6]:
for result in results[7:27]:
    ID = result.find_element_by_tag_name('td')
    print(ID.text)

3503598
1401575
5001797
2103723
2103914
4104757
0801306
3901432
3609624
3609931
1519799
4407379
4407296
0203332
2901986
0801417
4300768
4300776
2302283
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

In [7]:
rows = []
for result in results[7:27]:
    row = {}
    ID = result.find_element_by_tag_name('td')
    row['Operator ID'] = ID.text
    name = result.find_elements_by_tag_name('td')[2]
    row['Operator name'] = name.text
    mine = result.find_elements_by_tag_name('td')[3]
    row['Mine name'] = mine.text
    state = result.find_elements_by_tag_name('td')[1]
    row['State'] = state.text
    mine_type = result.find_elements_by_tag_name('td')[4]
    row['Mine type'] = mine_type.text
    coal_metal = result.find_elements_by_tag_name('td')[5]
    row['Coal or metal'] = coal_metal.text
    status = result.find_elements_by_tag_name('td')[6]
    row['Status'] = status.text
    commodity = result.find_elements_by_tag_name('td')[7]
    row['Commodity'] = commodity.text  
    rows.append(row)
print(rows)

[{'Operator ID': '3503598', 'Operator name': 'Newberg Rock & Dirt  ', 'Mine name': 'Newberg Rock & Dirt', 'State': 'OR ', 'Mine type': 'Surface', 'Coal or metal': 'M ', 'Status': 'Active ', 'Commodity': 'Crushed, Broken Stone NEC '}, {'Operator ID': '1401575', 'Operator name': 'Bender Sand & Dirt  ', 'Mine name': 'BENDER SAND & DIRT', 'State': 'KS ', 'Mine type': 'Surface', 'Coal or metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '5001797', 'Operator name': 'Dirt Company  ', 'Mine name': 'Bush Pilot', 'State': 'AK ', 'Mine type': 'Surface', 'Coal or metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '2103723', 'Operator name': 'Dirt Doctor Inc  ', 'Mine name': 'Rock Lake Plant', 'State': 'MN ', 'Mine type': 'Surface', 'Coal or metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '2103914', 'Operator name': 'Dirt Work Specialists

### Save that to a CSV named `dirt-operators.csv`

In [8]:
import pandas as pd
df = pd.DataFrame(rows)
df.to_csv('dirt-operators.csv', index=False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [9]:
df = pd.read_csv('dirt-operators.csv')
df.head()

,Coal or metal,Commodity,Mine name,Mine type,Operator ID,Operator name,State,Status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,3503598,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,BENDER SAND & DIRT,Surface,1401575,Bender Sand & Dirt,KS,Intermittent
2,M,Construction Sand and Gravel,Bush Pilot,Surface,5001797,Dirt Company,AK,Intermittent
3,M,Construction Sand and Gravel,Rock Lake Plant,Surface,2103723,Dirt Doctor Inc,MN,Intermittent
4,M,Construction Sand and Gravel,Astec Plant,Surface,2103914,Dirt Work Specialists LLC,MN,Intermittent
